In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from matplotlib import pyplot
from pandas import datetime
from pandas import concat

import pandas as pd
import numpy as np
import seaborn as sns

Using TensorFlow backend.


In [2]:
df = pd.read_csv('Avocado.csv',usecols=['Date','Total Volume', 'AveragePrice', 'Total Bags','Small Bags', 'Large Bags', 'XLarge Bags', 'type', 'year', 'region'])

In [3]:
df.sort_values(by=['Date'],inplace=True)
df.set_index("Date", inplace = True) 

#rearranging dataset 
df = df[['Total Volume', 'AveragePrice', 'Total Bags','Small Bags', 'Large Bags', 'XLarge Bags', 'type', 'year', 'region']]

In [4]:
df.head()

,Total Volume,AveragePrice,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
Date,,,,,,,,,
2015-01-04,27365.89,1.75,13598.46,13061.10,537.36,0.0,organic,2015,Southeast
2015-01-04,17723.17,1.49,905.55,905.55,0.00,0.0,organic,2015,Chicago
2015-01-04,2896.72,1.68,2528.08,2528.08,0.00,0.0,organic,2015,HarrisburgScranton
2015-01-04,54956.80,1.52,14925.18,11264.80,3660.38,0.0,conventional,2015,Pittsburgh
2015-01-04,1505.12,1.64,374.35,186.67,187.68,0.0,organic,2015,Boise


In [5]:
df.describe()

,Total Volume,AveragePrice,Total Bags,Small Bags,Large Bags,XLarge Bags,year
count,1.824900e+04,18249.000000,1.824900e+04,1.824900e+04,1.824900e+04,18249.000000,18249.000000
mean,8.506440e+05,1.405978,2.396392e+05,1.821947e+05,5.433809e+04,3106.426507,2016.147899
std,3.453545e+06,0.402677,9.862424e+05,7.461785e+05,2.439660e+05,17692.894652,0.939938
min,8.456000e+01,0.440000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2015.000000
25%,1.083858e+04,1.100000,5.088640e+03,2.849420e+03,1.274700e+02,0.000000,2015.000000
50%,1.073768e+05,1.370000,3.974383e+04,2.636282e+04,2.647710e+03,0.000000,2016.000000
75%,4.329623e+05,1.660000,1.107834e+05,8.333767e+04,2.202925e+04,132.500000,2017.000000
max,6.250565e+07,3.250000,1.937313e+07,1.338459e+07,5.719097e+06,551693.650000,2018.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18249 entries, 2015-01-04 to 2018-03-25
Data columns (total 9 columns):
Total Volume    18249 non-null float64
AveragePrice    18249 non-null float64
Total Bags      18249 non-null float64
Small Bags      18249 non-null float64
Large Bags      18249 non-null float64
XLarge Bags     18249 non-null float64
type            18249 non-null object
year            18249 non-null int64
region          18249 non-null object
dtypes: float64(6), int64(1), object(2)
memory usage: 1.4+ MB


split data to training and testing here: 
use date before 2018 as training and 2018 dates as testing 

scaled=scaler.fit(values)
#training= scaled.transform(training_data)
#test= scaled.transform(test_data)

In [7]:
for cat in ['type', 'region']:
    tmp = pd.get_dummies(df[cat], prefix=cat, drop_first=True)
    df.drop(cat, axis=1, inplace=True)
    df = pd.concat([df, tmp], axis=1)

In [8]:
df_train = df[df['year'] < 2018]
df_test = df[df['year'] >= 2018]

In [9]:
df.head()

,Total Volume,AveragePrice,Total Bags,Small Bags,Large Bags,XLarge Bags,year,type_organic,region_Atlanta,region_BaltimoreWashington,...,region_SouthCarolina,region_SouthCentral,region_Southeast,region_Spokane,region_StLouis,region_Syracuse,region_Tampa,region_TotalUS,region_West,region_WestTexNewMexico
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-04,27365.89,1.75,13598.46,13061.10,537.36,0.0,2015,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2015-01-04,17723.17,1.49,905.55,905.55,0.00,0.0,2015,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-04,2896.72,1.68,2528.08,2528.08,0.00,0.0,2015,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-04,54956.80,1.52,14925.18,11264.80,3660.38,0.0,2015,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-04,1505.12,1.64,374.35,186.67,187.68,0.0,2015,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18249 entries, 2015-01-04 to 2018-03-25
Data columns (total 61 columns):
Total Volume                  18249 non-null float64
AveragePrice                  18249 non-null float64
Total Bags                    18249 non-null float64
Small Bags                    18249 non-null float64
Large Bags                    18249 non-null float64
XLarge Bags                   18249 non-null float64
year                          18249 non-null int64
type_organic                  18249 non-null uint8
region_Atlanta                18249 non-null uint8
region_BaltimoreWashington    18249 non-null uint8
region_Boise                  18249 non-null uint8
region_Boston                 18249 non-null uint8
region_BuffaloRochester       18249 non-null uint8
region_California             18249 non-null uint8
region_Charlotte              18249 non-null uint8
region_Chicago                18249 non-null uint8
region_CincinnatiDayton       18249 non-null uint8
reg

In [11]:
y_train = df_train['Total Volume']
y_test = df_test['Total Volume']

In [12]:
x_train = df_train
x_train.drop('Total Volume', axis = 1)

,AveragePrice,Total Bags,Small Bags,Large Bags,XLarge Bags,year,type_organic,region_Atlanta,region_BaltimoreWashington,region_Boise,...,region_SouthCarolina,region_SouthCentral,region_Southeast,region_Spokane,region_StLouis,region_Syracuse,region_Tampa,region_TotalUS,region_West,region_WestTexNewMexico
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-04,1.75,13598.46,13061.10,537.36,0.00,2015,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2015-01-04,1.49,905.55,905.55,0.00,0.00,2015,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-04,1.68,2528.08,2528.08,0.00,0.00,2015,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-04,1.52,14925.18,11264.80,3660.38,0.00,2015,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-01-04,1.64,374.35,186.67,187.68,0.00,2015,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31,1.15,10309.11,5004.55,5304.56,0.00,2017,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-12-31,1.86,22469.27,20109.15,2360.12,0.00,2017,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-12-31,1.72,1292.72,213.18,1079.54,0.00,2017,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [13]:
x_test = df_test
x_test.drop('Total Volume', axis = 1)

,AveragePrice,Total Bags,Small Bags,Large Bags,XLarge Bags,year,type_organic,region_Atlanta,region_BaltimoreWashington,region_Boise,...,region_SouthCarolina,region_SouthCentral,region_Southeast,region_Spokane,region_StLouis,region_Syracuse,region_Tampa,region_TotalUS,region_West,region_WestTexNewMexico
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-07,1.25,8423.53,8423.53,0.00,0.00,2018,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-01-07,1.22,18452.29,16829.87,1622.42,0.00,2018,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-01-07,1.38,29847.95,28597.18,1250.77,0.00,2018,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-01-07,1.45,16811.19,14157.14,2654.05,0.00,2018,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-01-07,1.15,9816.36,9106.36,710.00,0.00,2018,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-03-25,1.36,127409.04,103579.41,22467.04,1362.59,2018,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-03-25,0.70,4014132.29,3398569.92,546409.74,69152.63,2018,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2018-03-25,1.42,129163.36,109052.26,20111.10,0.00,2018,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [14]:
from sklearn import preprocessing

# encoder = preprocessing.LabelEncoder()
# values[:,6] = encoder.fit_transform(values[:,6])
# values[:,8] = encoder.fit_transform(values[:,8])
# ensure all data is float
# values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

In [16]:
model = Sequential()
model.add(Dense(61, input_dim=61, kernel_initializer='normal', activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='relu'))

optimizer = Adam(lr=0.35)
model.compile(loss='mse', optimizer=optimizer, metrics=['mse', 'mae'])

In [17]:
model.fit(x_train, y_train, epochs=300, batch_size=5000, verbose=1)

Epoch 1/300
16953/16953 [==============================] - 1s 42us/step - loss: 12126257847153.9941 - mse: 12126257152000.0000 - mae: 833679.3750
Epoch 2/300
16953/16953 [==============================] - 0s 4us/step - loss: 12091630936717.7949 - mse: 12091630026752.0000 - mae: 825927.6875
Epoch 3/300
16953/16953 [==============================] - 0s 5us/step - loss: 11828504928872.1035 - mse: 11828504559616.0000 - mae: 830786.7500
Epoch 4/300
16953/16953 [==============================] - 0s 6us/step - loss: 11081781579470.3652 - mse: 11081781084160.0000 - mae: 1023540.0000
Epoch 5/300
16953/16953 [==============================] - 0s 5us/step - loss: 10153543938837.8203 - mse: 10153543860224.0000 - mae: 1467742.1250
Epoch 6/300
16953/16953 [==============================] - 0s 5us/step - loss: 8279746878134.5059 - mse: 8279746936832.0000 - mae: 1228719.5000
Epoch 7/300
16953/16953 [==============================] - 0s 5us/step - loss: 5144122784228.3057 - mse: 5144123015168.0000 - ma

16953/16953 [==============================] - 0s 6us/step - loss: 59201219161.9692 - mse: 59201216512.0000 - mae: 68007.1953
Epoch 60/300
16953/16953 [==============================] - 0s 5us/step - loss: 57247381861.4607 - mse: 57247379456.0000 - mae: 67151.4688
Epoch 61/300
16953/16953 [==============================] - 0s 4us/step - loss: 58954116769.6667 - mse: 58954113024.0000 - mae: 67401.9375
Epoch 62/300
16953/16953 [==============================] - 0s 5us/step - loss: 57262760060.6703 - mse: 57262759936.0000 - mae: 66795.6250
Epoch 63/300
16953/16953 [==============================] - 0s 5us/step - loss: 57470179827.2853 - mse: 57470181376.0000 - mae: 66860.4219
Epoch 64/300
16953/16953 [==============================] - 0s 5us/step - loss: 55695135728.2350 - mse: 55695134720.0000 - mae: 66148.0703
Epoch 65/300
16953/16953 [==============================] - 0s 5us/step - loss: 56360141783.3493 - mse: 56360144896.0000 - mae: 66123.0859
Epoch 66/300
16953/16953 [==============

16953/16953 [==============================] - 0s 4us/step - loss: 27932844433.0712 - mse: 27932844032.0000 - mae: 52465.9727
Epoch 118/300
16953/16953 [==============================] - 0s 4us/step - loss: 30521843805.6235 - mse: 30521841664.0000 - mae: 55814.8047
Epoch 119/300
16953/16953 [==============================] - 0s 4us/step - loss: 28615682474.8026 - mse: 28615682048.0000 - mae: 54782.4062
Epoch 120/300
16953/16953 [==============================] - 0s 4us/step - loss: 26864578926.2795 - mse: 26864578560.0000 - mae: 52690.9375
Epoch 121/300
16953/16953 [==============================] - 0s 5us/step - loss: 25974635076.2244 - mse: 25974636544.0000 - mae: 51701.4844
Epoch 122/300
16953/16953 [==============================] - 0s 4us/step - loss: 25882381711.1987 - mse: 25882380288.0000 - mae: 51944.0195
Epoch 123/300
16953/16953 [==============================] - 0s 4us/step - loss: 25229911149.9926 - mse: 25229912064.0000 - mae: 51544.6758
Epoch 124/300
16953/16953 [=======

16953/16953 [==============================] - 0s 6us/step - loss: 10060303663.5215 - mse: 10060304384.0000 - mae: 42483.5859
Epoch 176/300
16953/16953 [==============================] - 0s 5us/step - loss: 9780313476.9907 - mse: 9780314112.0000 - mae: 42353.1914
Epoch 177/300
16953/16953 [==============================] - 0s 5us/step - loss: 9071751984.2161 - mse: 9071752192.0000 - mae: 41272.1133
Epoch 178/300
16953/16953 [==============================] - 0s 6us/step - loss: 9262561305.6710 - mse: 9262561280.0000 - mae: 41347.4219A: 0s - loss: 9415254869.3333 - mse: 9415255040.0000 - mae: 41305.996
Epoch 179/300
16953/16953 [==============================] - 0s 5us/step - loss: 8859266119.7881 - mse: 8859267072.0000 - mae: 40526.6133
Epoch 180/300
16953/16953 [==============================] - 0s 5us/step - loss: 8655786315.1857 - mse: 8655785984.0000 - mae: 40894.3906
Epoch 181/300
16953/16953 [==============================] - 0s 5us/step - loss: 8368900594.4397 - mse: 8368900608.

16953/16953 [==============================] - 0s 5us/step - loss: 3911540287.6036 - mse: 3911540224.0000 - mae: 33434.3516
Epoch 234/300
16953/16953 [==============================] - 0s 5us/step - loss: 3881547122.5982 - mse: 3881547008.0000 - mae: 33543.0664
Epoch 235/300
16953/16953 [==============================] - 0s 5us/step - loss: 3877548625.5431 - mse: 3877548544.0000 - mae: 33353.4219
Epoch 236/300
16953/16953 [==============================] - 0s 5us/step - loss: 3799139916.4240 - mse: 3799140096.0000 - mae: 33382.0938
Epoch 237/300
16953/16953 [==============================] - 0s 5us/step - loss: 3721958165.9864 - mse: 3721958144.0000 - mae: 32900.5391
Epoch 238/300
16953/16953 [==============================] - 0s 5us/step - loss: 3704353550.0586 - mse: 3704353792.0000 - mae: 32965.9062
Epoch 239/300
16953/16953 [==============================] - 0s 4us/step - loss: 3654406905.5370 - mse: 3654406912.0000 - mae: 32763.3750
Epoch 240/300
16953/16953 [=====================

16953/16953 [==============================] - 0s 6us/step - loss: 2378638994.2943 - mse: 2378638848.0000 - mae: 28456.9551
Epoch 293/300
16953/16953 [==============================] - 0s 5us/step - loss: 2291448970.3816 - mse: 2291449088.0000 - mae: 27947.5176
Epoch 294/300
16953/16953 [==============================] - 0s 5us/step - loss: 2280865577.3303 - mse: 2280865536.0000 - mae: 27960.7578
Epoch 295/300
16953/16953 [==============================] - 0s 5us/step - loss: 2282218460.3475 - mse: 2282218496.0000 - mae: 27927.6855
Epoch 296/300
16953/16953 [==============================] - 0s 5us/step - loss: 2189563106.4633 - mse: 2189563136.0000 - mae: 27579.8398
Epoch 297/300
16953/16953 [==============================] - 0s 5us/step - loss: 2167260562.7776 - mse: 2167260416.0000 - mae: 27257.6230
Epoch 298/300
16953/16953 [==============================] - 0s 5us/step - loss: 2170256416.4511 - mse: 2170256640.0000 - mae: 27362.5195
Epoch 299/300
16953/16953 [=====================

In [18]:
from sklearn import metrics

def get_metrics(reg, y_test, y_pred):
    res = []
    
    res.append(str(reg.__class__.__name__))
    mae = metrics.mean_absolute_error(y_test, y_pred)
    res.append(mae)
    mse = metrics.mean_squared_error(y_test, y_pred)
    res.append(mse)
    rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    res.append(rmse)
    r2 = metrics.r2_score(y_test, y_pred)
    res.append(r2)
    
    return res
               
def show_metrics(res):
    print(res[0])
    print(' MAE : ', res[1])
    print(' MSE : ', res[2])
    print(' RMSE: ', res[3])
    print(' R^2 : ', res[4])

In [19]:
m = get_metrics(model, model.predict(x_test), y_test)
show_metrics(m)

Sequential
 MAE :  40920.20411810982
 MSE :  5359595953.718874
 RMSE:  73209.26139307016
 R^2 :  0.9997100209553621
